##### The below code reads the last frame of the video and appends it back to the video at the end for a specified duration

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
def append_last_frame(video_path, output_path, duration=3):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    width  = int(cap.get(3))  # Frame width
    height = int(cap.get(4))  # Frame height
    fps    = cap.get(5)       # Frames per second

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set the frame position to the last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

    # Read the last frame
    ret, last_frame = cap.read()

    # Check if the frame is None (indicating an issue with frame retrieval)
    if not ret or last_frame is None:
        raise ValueError("Error reading the last frame from the video.")

    # Release the video capture object
    cap.release()

    # Repeat the last frame to create an array for the extended duration
    repeated_frames = np.tile(last_frame, (int(duration * fps), 1, 1, 1))

    # Open the original video file for reading
    cap = cv2.VideoCapture(video_path)

    # Open a new video file for writing, using the same codec as the original video
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Read and write frames from the original video
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if the video is finished

        out.write(frame)

    # Write the repeated frames to the output video file
    for repeated_frame in repeated_frames:
        out.write(repeated_frame)

    # Release the video capture and writer objects
    cap.release()
    out.release()

In [5]:
if __name__ == "__main__":
    stimulus_videos_directory = "../../stimulus_dataset/stimulus_preview/cut_preview/"
    output_directory = "../../stimulus_dataset/stimulus_preview/final_cut_preview/"

    if not os.path.exists(output_directory):
        os.mkdir(output_directory)

    files_to_ignore = ['.DS_Store']

    videos = [video for video in os.listdir(stimulus_videos_directory) if video not in files_to_ignore]

    for video in tqdm(videos, desc="Processing Video", unit="video"):
        video_path = os.path.join(stimulus_videos_directory, video)
        output_path = os.path.join(output_directory, f"preview_{video.split('.')[0].strip()}_final.mp4")
        append_last_frame(video_path, output_path, duration=3)

Processing Video: 100%|████████████████████████| 5/5 [00:08<00:00,  1.79s/video]
